In [80]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [81]:
import os
import selenium
import time
from selenium import webdriver
from bs4 import BeautifulSoup as BS
from selenium.webdriver.chrome.options import Options
import sys
import pickle
import pandas as pd
from tqdm import tqdm, tqdm_notebook
from Codes.Review_scrawler import *
from Codes.feature_extraction import *

### Load the descriptions

In [26]:
with open('product_names.txt', "r") as x:
    all_product = x.read().splitlines()

In [39]:
product_name_description = Scrawl_product_description('product_names.txt')

100%|██████████| 1/1 [2:00:00<00:00, 7200.77s/it]


In [43]:
with open('product_description.pickle', 'wb') as handle:
    pickle.dump(product_name_description, handle, pickle.HIGHEST_PROTOCOL)

In [67]:
product_name_description = pickle.load(open('product_description.pickle', 'rb'))


In [68]:

sum_products = 0
for key in tqdm(product_name_description.keys()):
    sum_products += len(product_name_description[key])

100%|██████████| 455/455 [00:00<00:00, 608936.92it/s]


### The list of POS tags is as follows, with examples of what each POS stands for.

* CC coordinating conjunction
* CD cardinal digit
* DT determiner
* EX existential there (like: “there is” … think of it like “there exists”)
* FW foreign word
* IN preposition/subordinating conjunction
* JJ adjective ‘big’
* JJR adjective, comparative ‘bigger’
* JJS adjective, superlative ‘biggest’
* LS list marker 1)
* MD modal could, will
* NN noun, singular ‘desk’
* NNS noun plural ‘desks’
* NNP proper noun, singular ‘Harrison’
* NNPS proper noun, plural ‘Americans’
* PDT predeterminer ‘all the kids’
* POS possessive ending parent’s
* PRP personal pronoun I, he, she
* PRP possessive pronoun my, his, hers
* RB adverb very, silently,
* RBR adverb, comparative better
* RBS adverb, superlative best
* RP particle give up
* TO, to go ‘to’ the store.
* UH interjection, errrrrrrrm
* VB verb, base form take
* VBD verb, past tense took
* VBG verb, gerund/present participle taking
* VBN verb, past participle taken
* VBP verb, sing. present, non-3d take
* VBZ verb, 3rd person sing. present takes
* WDT wh-determiner which
* WP wh-pronoun who, what
* WP possessive wh-pronoun whose
* WRB wh-abverb where, when

## Feature Extraction - Basic Implementation


* __Step 1: Finding Candidate Features:__ First approach for extracting features was based on the intuition that product features are usually nouns or noun phrases [1]. Using the POS associated with words during the data generation phase, we created a new view of data where each sentence in the review was considered  as a bag of words. The  words  chosen  to  represent  a  sentence  were  those  that  were  marked  as  nouns  (NN/NNS).  We chose  to  ignore  proper  nouns,  which  we  believe  cannot  be  features  associated  with  a  product. 

* __Step 2: Finding Candidate Features:__ Furthermore, as we observed in the data, some phrases that represent features (such as optical zoom) were made of two classes of words, nouns and adjectives. So as to be able to detect such features, we also included words marked as adjectives (JJ/JJR/JJS) in our bag of words model.

* __Step 3: Finding Frequent Features:__ The  next  step  was  to  extract  frequent  features  from  the  candidate  feature  words.  We  used  an implementation  based  on  the  APRIORI  algorithm  [10]  for  identifying  frequently  occurring word/word  pairs  from  the  bag  of  words  data  model. Using  a  support  threshold  of  0.5%,  we  got  a good  set  of  candidate  features.  

* __Step 4: Remove Opinion Words:__ However,  we  observed  that  because  of  inclusion  of  adjectives  when finding the frequent item sets, we got many candidate features that actually were opinion words (like good,  best,  bad).  In  order  to  filter  out  such  frequent  items,  we  considered  only  those  single  items that  occurred  as a  noun  somewhere in  the  corpus, or  in  case  of  word phrases, if  at  least one  of the word in the phrase occurred as a noun somewhere in the corpus. 

* __Step 5: Feature Set:__ The resulting set of words was our feature set for the product trained on.

### Step 1 through 3

In [ ]:
model = Features(product_name_description)
candidate_featres, pos = model.get_candidate_attributes(show = True, lower = True)


In [85]:
import numpy as np

In [88]:
ramin = np.random.choice(pos, 100)
jade = np.random.choice(pos, 100)

In [93]:
np.save('jade_pos.npy', jade)
np.save('ramin_pos.npy', ramin)

In [73]:
frequency_counts = model.get_frequent_attributes(support = 0.005)

[list([('tongue', 'NN'), ('logo', 'NN'), ('and', 'CC'), ('exaggerated', 'JJ'), ('tongue', 'NN'), ('top', 'JJ'), ('reminiscent', 'NN'), ('of', 'IN'), ('the', 'DT'), ('original', 'JJ'), ('air', 'NN'), ('max', 'VBD'), ('180', 'CD')])
 list([('the', 'DT'), ('men', 'NNS'), ("'s", 'POS'), ('nike', 'NN'), ('lebron', 'VBD'), ('16', 'CD'), ('basketball', 'NN'), ('shoes', 'NNS'), ('deliver', 'VB'), ('a', 'DT'), ('powerful', 'JJ'), ('combination', 'NN'), ('of', 'IN'), ('support', 'NN')])
 list([('a', 'DT'), ('fusion', 'NN'), ('of', 'IN'), ('the', 'DT'), ('best', 'JJS'), ('adidas', 'NN'), ('technologies', 'NNS'), ('and', 'CC'), ('progressive', 'JJ'), ('street', 'NN'), ('style', 'NN'), ('collide', 'NN'), ('for', 'IN'), ('the', 'DT'), ('men', 'NNS'), ("'s", 'POS'), ('adidas', 'NNS'), ('nmd', 'CC'), ('runner', 'NN'), ('r1', 'VBP'), ('casual', 'JJ'), ('shoes', 'NNS'), ('.', '.'), ('if', 'IN'), ('you', 'PRP'), ('are', 'VBP'), ('looking', 'VBG'), ('for', 'IN'), ('the', 'DT'), ('perfect', 'JJ'), ('kicks'